In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
df_train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
df_test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

In [ ]:
df_train.head()

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
xlmr = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli")
tokenizer =AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")

In [ ]:
# check model classes name and dataset classses name are same or not
xlmr.config.label2id

In [ ]:
# in kaggle dataset entitlement 0 and contradiiction 2
df_train["label"] = df_train["label"].replace([0,2],[2,0])

In [ ]:
df_train.head(3)

### EDA on dataset

In [ ]:
# distribution of training data based on language
train_data_lang = df_train.groupby("language").count().reset_index()[["language","id"]]
# plot pie chart
plt.figure(figsize=(10,10))
plt.pie(train_data_lang["id"],labels = train_data_lang["language"],autopct = "%1.1f%%")
plt.title("distribution of train data based on language")
plt.show()


### sample data creation

In [ ]:
df_train.shape

In [ ]:
# # sample data created  be same distribution of text in training data
train_data_lang["sample_count"] = train_data_lang["id"]/10  # sample data
sample_train_data = pd.DataFrame(columns =df_train.columns)
for i in range(len(train_data_lang)):
    df = df_train[df_train["language"]==train_data_lang["language"][i]]
    n = int(train_data_lang["sample_count"][i])
    df = df.sample(n).reset_index(drop =True)
    sample_train_data = sample_train_data.append(df)
    sample_train_data = sample_train_data.reset_index(drop =True)
#plot distribution of sample data based on language
sample_train_data_lang = sample_train_data.groupby("language").count().reset_index()[["language","id"]]
plt.figure(figsize=(10,10))
plt.pie(sample_train_data_lang["id"],labels = sample_train_data_lang["language"],autopct = "%1.1f%%")
plt.title("distribution of sample train data based on language")
plt.show()

### function to get prediction from xlmr model

In [ ]:

def get_tokens_xlmr_model(data):
    """ function which create tokens for the passed data using xlmr model
    input -- dataframe 
    output -- list of tokens"""
    batch_tokens = []
    for i in range(len(data)):
        tokens = tokenizer.encode(data["premise"][i],data["hypothesis"][i],return_tensors = "pt",truncation_strategy="only_first")
        batch_tokens.append(tokens)
    return batch_tokens
def get_predicts_xlmr_model(tokens):
    """function which creates prediction for the passed tokens using xlmr model
    input -- list of tokens.  output--list of predictions"""
    batch_predict = []
    for i in tokens:
        predict = xlmr(i)[0][0]
        predict =int(predict.argmax())
        batch_predict.append(predict)
    return batch_predict

### prediction on sampled data

In [ ]:

sample_train_data_tokens = get_tokens_xlmr_model(sample_train_data)
sample_train_data_prediction = get_predicts_xlmr_model(sample_train_data_tokens)

### find the model accuracy on prediction

In [ ]:
import seaborn as sns
from sklearn.metrics import classification_report
sample_train_data["label"]= sample_train_data["label"].astype(str).astype(int)
x= np.array(sample_train_data["label"])
y= np.array(sample_train_data_prediction)
cm = np.zeros((3,3),dtype=int)
np.add.at(cm,[x,y],1)
sns.heatmap(cm,cmap="YlGnBu",annot =True, annot_kws={"size":16},fmt="g",xticklabels =["contradiction","neutral","entailment"],yticklabels =["contradiction","neutral","entailment"])
matrix = classification_report(x,y,labels=[0,1,2],target_names=["contradiction","neutral","entailment"])
print("classification report :\n",matrix)

### find accuracy at language level

In [ ]:
# sample_train_data[‘prediction’] = sample_train_data_predictions 
# sample_train_data[‘true_prediction’] = np.where(sample_train_data[‘label’]==sample_train_data[‘prediction’], 1, 0) 
# sample_train_data_predicted_lang = sample_train_data.groupby(‘language’).agg({‘id’:’count’, ‘true_prediction’:’sum’}).reset_index()[[‘language’,’id’,’true_prediction’]] 
# sample_train_data_predicted_lang[‘accuracy’] = round(sample_train_data_predicted_lang[‘true_prediction’]/sample_train_data_predicted_lang[‘id’], 2)
# sample_train_data_predicted_lang = sample_train_data_predicted_lang.sort_values(by=[‘accuracy’],ascending=False) 
# sample_train_data_predicted_lang
sample_train_data["prediction"]=sample_train_data_prediction
sample_train_data["true_prediction"]=np.where(sample_train_data["label"]==sample_train_data["prediction"],1,0)

In [ ]:
sample_train_data_predicted_lang =sample_train_data.groupby("language").agg({"id":"count","true_prediction":"sum"}).reset_index()[["language","id","true_prediction"]]
sample_train_data_predicted_lang["accuracy"]=round(sample_train_data_predicted_lang["true_prediction"]/sample_train_data_predicted_lang["id"],2)

In [ ]:
sample_train_data_predicted_lang= sample_train_data_predicted_lang.sort_values(by=["accuracy"],ascending=False)
sample_train_data_predicted_lang

### accuracy of roberta model